<a href="https://colab.research.google.com/github/voshna123/tensorflow_practice/blob/main/foodrecog_big_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2025-01-10 01:35:49--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-01-10 01:35:50 (25.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [16]:
from helper_functions import create_tensorboard_callback, plot_loss_curves,compare_historys

## Loading 101Food Dataset

In [2]:
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

--2025-01-10 01:16:55--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2025-01-10 01:16:55--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  21.7MB/s    in 3m 55s  

2025-01-10 01:20:51 (20.3 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]



In [7]:
# importing the "tarfile" module
import tarfile

# open file
file = tarfile.open('/content/food-101.tar.gz')

# extracting file
file.extractall()

file.close()

## plot the img

In [12]:
import os
import shutil
import random
from pathlib import Path
from typing import Dict

def split_nested_folders(
    source_folder: str,
    train_folder: str,
    test_folder: str,
    split_ratio: float = 0.8,
    random_seed: int = 42
) -> Dict[str, Dict[str, int]]:
    """
    Split a nested folder structure into training and test sets while preserving hierarchy.

    Args:
        source_folder: Path to the root folder containing nested folders
        train_folder: Path where training files will be copied
        test_folder: Path where test files will be copied
        split_ratio: Proportion of files to use for training (default: 0.8)
        random_seed: Random seed for reproducibility (default: 42)

    Returns:
        Dict containing statistics for each subfolder
    """
    random.seed(random_seed)

    # Convert to Path objects
    source_path = Path(source_folder)
    train_path = Path(train_folder)
    test_path = Path(test_folder)

    # Create output root directories
    train_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)

    stats = {}

    # Walk through all subdirectories
    for root, dirs, files in os.walk(source_path):
        if not files:
            continue

        # Get the relative path from source root
        rel_path = Path(root).relative_to(source_path)

        # Create corresponding directories in train and test
        train_subdir = train_path / rel_path
        test_subdir = test_path / rel_path

        train_subdir.mkdir(parents=True, exist_ok=True)
        test_subdir.mkdir(parents=True, exist_ok=True)

        # Shuffle files
        random.shuffle(files)

        # Calculate split point
        split_idx = int(len(files) * split_ratio)
        train_files = files[:split_idx]
        test_files = files[split_idx:]

        # Copy files to respective directories
        for file in train_files:
            src_file = Path(root) / file
            dst_file = train_subdir / file
            shutil.copy2(src_file, dst_file)

        for file in test_files:
            src_file = Path(root) / file
            dst_file = test_subdir / file
            shutil.copy2(src_file, dst_file)

        # Store statistics
        stats[str(rel_path)] = {
            'train': len(train_files),
            'test': len(test_files),
            'total': len(files)
        }

    # Calculate total statistics
    total_stats = {
        'train': sum(s['train'] for s in stats.values()),
        'test': sum(s['test'] for s in stats.values()),
        'total': sum(s['total'] for s in stats.values())
    }
    stats['total'] = total_stats

    return stats

In [13]:
stats = split_nested_folders("food-101/images",
                                     "train",
                                     "test")

ValueError: too many values to unpack (expected 2)

In [14]:
import tensorflow as tf

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/train",
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=32,
    shuffle = True,
    seed = 42
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/test",
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=32,
    shuffle = False,
    seed = 42
)


Found 80800 files belonging to 101 classes.
Found 20200 files belonging to 101 classes.


## Preprocessing data

In [17]:
train_data = train_data.prefetch(tf.data.AUTOTUNE)
test_data = test_data.prefetch(tf.data.AUTOTUNE)

In [18]:
train_data, test_data

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 101), dtype=tf.float32, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 101), dtype=tf.float32, name=None))>)

## Creating chck callback

In [19]:
chck_path ="chck.weights.h5"

chck = tf.keras.callbacks.ModelCheckpoint(chck_path,
                                          monitor = 'val_accuracy',
                                          save_best_only = True,
                                          save_weights_only = True)

## setup mixed precision

In [21]:
## this must be run on at least T4 GPU
tf.keras.mixed_precision.set_global_policy('mixed_float16')

## build model

In [22]:
base = tf.keras.applications.EfficientNetB0(include_top = False)
base.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [32]:
Inputs = tf.keras.layers.Input(shape = (224,224,3),name = "input_layer")
x = base(Inputs,training = False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(101,activation = "softmax", dtype = tf.float32)(x)

model = tf.keras.Model(Inputs,outputs)

In [33]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ["accuracy"])

In [39]:
for x in model.layers:
  print(x.name, x.dtype, x.trainable, x.dtype_policy)

input_layer float32 True <DTypePolicy "mixed_float16">
efficientnetb0 float32 False <DTypePolicy "mixed_float16">
global_average_pooling2d_6 float32 True <DTypePolicy "mixed_float16">
dense float32 True <DTypePolicy "float32">


In [40]:
for x in model.layers[1].layers:
  print(x.name, x.dtype, x.trainable, x.dtype_policy)

input_layer float32 False <DTypePolicy "mixed_float16">
rescaling float32 False <DTypePolicy "mixed_float16">
normalization float32 False <DTypePolicy "mixed_float16">
rescaling_1 float32 False <DTypePolicy "mixed_float16">
stem_conv_pad float32 False <DTypePolicy "mixed_float16">
stem_conv float32 False <DTypePolicy "mixed_float16">
stem_bn float32 False <DTypePolicy "mixed_float16">
stem_activation float32 False <DTypePolicy "mixed_float16">
block1a_dwconv float32 False <DTypePolicy "mixed_float16">
block1a_bn float32 False <DTypePolicy "mixed_float16">
block1a_activation float32 False <DTypePolicy "mixed_float16">
block1a_se_squeeze float32 False <DTypePolicy "mixed_float16">
block1a_se_reshape float32 False <DTypePolicy "mixed_float16">
block1a_se_reduce float32 False <DTypePolicy "mixed_float16">
block1a_se_expand float32 False <DTypePolicy "mixed_float16">
block1a_se_excite float32 False <DTypePolicy "mixed_float16">
block1a_project_conv float32 False <DTypePolicy "mixed_float16"

In [ ]:
hist = model.fit(train_data,
          epochs = 5,
          validation_data = test_data,
          validation_steps = int(0.15 * len(test_data)),
          callbacks = [chck])

Epoch 1/5
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 206s 81ms/step - accuracy: 0.5707 - loss: 1.7513 - val_accuracy: 0.5918 - val_loss: 1.5636
Epoch 2/5
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 243s 74ms/step - accuracy: 0.6566 - loss: 1.3333 - val_accuracy: 0.6200 - val_loss: 1.5039
Epoch 3/5
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 181s 66ms/step - accuracy: 0.6862 - loss: 1.2007 - val_accuracy: 0.6951 - val_loss: 1.1957
Epoch 4/5
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 165s 65ms/step - accuracy: 0.7058 - loss: 1.1175 - val_accuracy: 0.6439 - val_loss: 1.3744
Epoch 5/5
 934/2525 ━━━━━━━━━━━━━━━━━━━━ 1:38 62ms/step - accuracy: 0.7217 - loss: 1.0448

In [ ]:
model.evaluate(test_data)